## BERT fine-tuning + hyperparameter tuning

#### Setup

In [1]:
!pip install -q optuna
!pip install -q evaluate
!pip install -q emoji==0.6.0

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch import nn
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import (
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainerCallback,
    EarlyStoppingCallback
)
import evaluate
import torch.nn.functional as F
import optuna
import wandb

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
wandb.login(key="<wandb key>")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mayachn3 (mayachn3-maya-bondar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Data Preparation
### Load Dataset

In [6]:
train = pd.read_csv("OOT_train.csv", encoding='latin-1')
val = pd.read_csv("OOT_val.csv", encoding='latin-1')
test = pd.read_csv("OOT_test.csv", encoding='latin-1')


In [7]:
# train = train.head(1000)
# val = val.head(1000)
# test = test.head(1000)

### Preprocessing

In [8]:
#encoding the labels numerically from Sentiment

ordinal_mapping = {
    'Extremely Negative': 0,
    'Negative': 1,
    'Neutral': 2,
    'Positive': 3,
    'Extremely Positive': 4
}

# map to ordinal labels
train["ordinal_label_id"] = train["Sentiment"].map(ordinal_mapping)
val["ordinal_label_id"] = val["Sentiment"].map(ordinal_mapping)
test["ordinal_label_id"] = test["Sentiment"].map(ordinal_mapping)



In [9]:
# Concat the relevant columns into one string with seperation.
# for example: "Tweet: my food stock is low | Location: Canada | Date: 2020-03-17 | URL: https://t.co/abcd"

# Function to build the input string from multiple columns
def build_augmented_input(row):
    parts = []

    if pd.notna(row.get('clean_tweet')):
        parts.append(f"{row['clean_tweet']}")

    if pd.notna(row.get('Location_standardized')) and row['Location_standardized'].lower() != 'unknown':
        parts.append(f"{row['Location_standardized']}")

    if pd.notna(row.get('TweetAt')):
        parts.append(f"{row['TweetAt']}")


    return " | ".join(parts)

# Apply to the DataFrames
train['model_input'] = train.apply(build_augmented_input, axis=1)
val['model_input'] = val.apply(build_augmented_input, axis=1)
test['model_input'] = test.apply(build_augmented_input, axis=1)

# Create  new DataFrames with only what's needed for modeling
formatted_train = train[['model_input', 'ordinal_label_id']].copy()
formatted_val = val[['model_input', 'ordinal_label_id']].copy()
formatted_test = test[['model_input', 'ordinal_label_id']].copy()



In [10]:
def balance_dataset(df, target_samples_per_class=5000):
    """Balance dataset by undersampling"""
    balanced_dfs = []

    print("Original class distribution:")
    print(df['ordinal_label_id'].value_counts().sort_index())

    for class_id in range(5):
        class_data = df[df['ordinal_label_id'] == class_id]

        if len(class_data) > target_samples_per_class:
            class_data = class_data.sample(n=target_samples_per_class, random_state=42)
            print(f"Class {class_id}: {len(class_data)} samples (undersampled)")
        else:
            print(f"Class {class_id}: {len(class_data)} samples (kept all)")

        balanced_dfs.append(class_data)

    balanced_df = pd.concat(balanced_dfs, ignore_index=True).sample(frac=1, random_state=42)

    print(f"Balanced dataset: {len(balanced_df)} total samples")
    print("New distribution:")
    print(balanced_df['ordinal_label_id'].value_counts().sort_index())

    return balanced_df

# Apply balancing to training data
formatted_train = balance_dataset(formatted_train, target_samples_per_class=5000)

Original class distribution:
ordinal_label_id
0     5175
1     9230
2     6784
3    10140
4     5845
Name: count, dtype: int64
Class 0: 5000 samples (undersampled)
Class 1: 5000 samples (undersampled)
Class 2: 5000 samples (undersampled)
Class 3: 5000 samples (undersampled)
Class 4: 5000 samples (undersampled)
Balanced dataset: 25000 total samples
New distribution:
ordinal_label_id
0    5000
1    5000
2    5000
3    5000
4    5000
Name: count, dtype: int64


### Tokenization

## Model
### Define Model

In [11]:
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_data(data, max_length=128):
    return tokenizer(
        data['model_input'].tolist(),
        truncation=True,
        padding=False,
        max_length=max_length,
        add_special_tokens=True,
        return_attention_mask=True,
        return_token_type_ids=False
    )

train_encodings = tokenize_data(formatted_train)
val_encodings = tokenize_data(formatted_val)
test_encodings = tokenize_data(formatted_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
## define a PyTorch Dataset
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels  # Should be integers

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # For training
        return item

    def __len__(self):
        return len(self.labels)

# Convert labels to integers if not already
train_labels = formatted_train['ordinal_label_id'].tolist()
val_labels = formatted_val['ordinal_label_id'].tolist()
test_labels = formatted_test['ordinal_label_id'].tolist()


train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, test_labels)


In [13]:
# define mapping between label id and sentiment for later use and conveniency
ordinal_label2id = ordinal_mapping
ordinal_id2label = {v: k for k, v in ordinal_mapping.items()}

In [14]:
def compute_detailed_metrics(eval_pred):
    """Enhanced metrics using HuggingFace Evaluate library"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Load HuggingFace metrics (cached after first load)
    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")

    # Compute standard classification metrics
    results = {}

    # Basic metrics
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average='macro'))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average='weighted'))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average='macro'))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average='macro'))

    # Per-class F1 scores (HF doesn't have this built-in, so keep custom)
    f1_per_class = f1_score(labels, predictions, average=None)
    for i, class_name in enumerate(['extremely_negative', 'negative', 'neutral', 'positive', 'extremely_positive']):
        results[f'f1_{class_name}'] = f1_per_class[i]

        # Per-class precision and recall
        precision_per_class = precision_score(labels, predictions, average=None, zero_division=0)
        recall_per_class = recall_score(labels, predictions, average=None, zero_division=0)
        results[f'precision_{class_name}'] = precision_per_class[i]
        results[f'recall_{class_name}'] = recall_per_class[i]

        # Per-class accuracy
        class_mask = (labels == i)
        if class_mask.sum() > 0:
            results[f'accuracy_{class_name}'] = accuracy_score(labels[class_mask], predictions[class_mask])
        else:
            results[f'accuracy_{class_name}'] = 0.0

    # Custom ordinal metrics (HF doesn't have these)
    results['mae'] = np.mean(np.abs(predictions - labels))
    results['adjacent_accuracy'] = np.sum(np.abs(predictions - labels) <= 1) / len(labels)

    # Quadratic Weighted Kappa (custom)
    from sklearn.metrics import cohen_kappa_score
    try:
        qwk = cohen_kappa_score(labels, predictions, weights='quadratic')
        results['quadratic_weighted_kappa'] = qwk
    except:
        results['quadratic_weighted_kappa'] = 0.0

    return results

In [15]:
def find_optimal_batch_size(base_batch_size):
    """Find the largest batch size that fits in GPU memory"""
    if device.type == "cpu":
        return base_batch_size

    # Try larger batch sizes for GPU
    for multiplier in [4, 3, 2, 1]:
        try_batch_size = base_batch_size * multiplier
        try:
            # Test if this batch size fits
            dummy_input = torch.randn(try_batch_size, 128, 768, device=device)
            dummy_output = torch.randn(try_batch_size, 5, device=device)
            del dummy_input, dummy_output
            torch.cuda.empty_cache() if device.type == "cuda" else None
            return try_batch_size
        except RuntimeError:  # Out of memory
            continue
    return base_batch_size

In [16]:
class SimpleMetricsLogger(TrainerCallback):
    """Simple callback to log detailed metrics every epoch"""

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None or not wandb.run:
            return

        # Only log when we have evaluation metrics (after each epoch)
        if 'eval_loss' in logs:
            current_epoch = int(state.epoch)

            # Get current learning rate
            current_lr = args.learning_rate
            if 'learning_rate' in logs:
                current_lr = logs['learning_rate']

            #Get training loss from state history
            train_loss = 0
            if state.log_history:
                # Find the most recent training loss
                for log_entry in reversed(state.log_history):
                    if 'train_loss' in log_entry:
                        train_loss = log_entry['train_loss']
                        break

            detailed_metrics = {
                "Epoch": current_epoch,
                "Stage": 1,
                "Unfrozen_Layers": 12,
                "Train Loss": train_loss,
                "Train Accuracy": 0,  # Usually not computed during training
                "Validation Loss": logs.get('eval_loss', 0),
                "Validation Accuracy": logs.get('eval_accuracy', 0),
                "Validation Precision": logs.get('eval_precision_macro', 0),
                "Validation Recall": logs.get('eval_recall_macro', 0),
                "Validation F1": logs.get('eval_f1_macro', 0),
                "Validation MAE": logs.get('eval_mae', 0),
                "Validation Adjacent Accuracy": logs.get('eval_adjacent_accuracy', 0),
                "Validation QWK": logs.get('eval_quadratic_weighted_kappa', 0),
                "Learning_Rate": current_lr,
            }

            # Log to WandB
            wandb.log(detailed_metrics)

            #Print progress to console
            print(f"Epoch {current_epoch}: "
                  f"Train Loss: {train_loss:.4f}, "  # ← Now shows real values
                  f"Val Loss: {logs.get('eval_loss', 0):.4f}, "
                  f"Val F1: {logs.get('eval_f1_macro', 0):.4f}, "
                  f"QWK: {logs.get('eval_quadratic_weighted_kappa', 0):.4f}")

In [17]:
def save_training_checkpoint(model, optimizer, epoch, loss, trial_params, filepath, trial_number, current_score, trainer):
    """Save complete training checkpoint and handle best model updates"""
    global best_score, best_model_path

    # Get the trial directory from filepath
    trial_dir = os.path.dirname(filepath)

    # Save trial checkpoint
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        # 'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'trial_params': trial_params,
        'model_config': model.config.to_dict(),
        'tokenizer_name': model_name,
        'current_score': current_score,  # Add score to checkpoint
        'trial_number': trial_number,
    }

    # Save all trial files in the same directory
    torch.save(checkpoint, filepath)
    torch.save(model.state_dict(), os.path.join(trial_dir, 'model_bert_weights.pt'))
    torch.save(model, os.path.join(trial_dir, 'model_bert.pt'))

    print(f"✅ Trial checkpoint saved: {filepath}")
    print(f"✅ Model files saved in: {trial_dir}")

    # Update best model if needed
    if current_score > best_score:
        best_score = current_score

        # Save HuggingFace format to best model directory
        trainer.save_model(best_model_path)

        # Also save our custom format in best model directory
        os.makedirs(best_model_path, exist_ok=True)
        best_checkpoint_path = os.path.join(best_model_path, 'best_checkpoint.ckpt')
        best_weights_path = os.path.join(best_model_path, 'model_bert_weights.pt')
        best_model_file_path = os.path.join(best_model_path, 'model_bert.pt')

        torch.save(checkpoint, best_checkpoint_path)
        torch.save(model.state_dict(), best_weights_path)
        torch.save(model, best_model_file_path)

        print(f"🏆 New best model saved! Score: {current_score:.4f} (Trial {trial_number})")
        print(f"🏆 Best model files saved in: {best_model_path}")

        # Optional: Log to W&B
        # wandb.log({
        #     "best_score_so_far": current_score,
        #     "best_trial_number": trial_number,
        # })
    else:
        print(f"📊 Trial {trial_number} score: {current_score:.4f} (Best: {best_score:.4f})")

In [18]:
# Global variables to track best model
best_score = 0.0
best_model_path = "./best_bert_model_so_far"

### Hyperparameter Tuning

In [19]:
def objective(trial):
    """Clean, organized objective function for Optuna hyperparameter optimization"""
    global best_score, best_model_path

    # === GPU MEMORY CLEANUP ===
    if device.type == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

    # === HYPERPARAMETER SAMPLING ===
    # Core training parameters
    learning_rate = trial.suggest_float("learning_rate", 3e-5, 5e-4, log=True)
    base_batch_size = trial.suggest_categorical("batch_size", [16,32,64])
    label_smoothing = trial.suggest_float("label_smoothing", 0.05, 0.15)
    num_epochs = trial.suggest_int("num_epochs", 10, 15)

    # Advanced optimization parameters
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.05, 0.15)
    weight_decay = trial.suggest_float("weight_decay", 0.05, 0.15)

    # Model architecture parameters
    attention_dropout = trial.suggest_float("attention_dropout", 0.3, 0.4)
    hidden_dropout = trial.suggest_float("hidden_dropout", 0.3, 0.4)

    #R-drop parameter
    # rdrop_alpha = trial.suggest_float("rdrop_alpha", 0.0, 1.0)

    # PRINT CHOSEN PARAMETERS
    print(f"TRIAL {trial.number} - TESTING THESE PARAMETERS:")
    print(f"Learning Rate:      {learning_rate:.2e}")
    print(f"Epochs:             {num_epochs}")
    print(f"Warmup Ratio:       {warmup_ratio:.3f}")
    print(f"Weight Decay:       {weight_decay:.3f}")
    print(f"Attention Dropout:  {attention_dropout:.3f}")
    print(f"Hidden Dropout:     {hidden_dropout:.3f}")


    # Optimize batch size for available hardware
    batch_size = find_optimal_batch_size(base_batch_size)


    # === EXPERIMENT TRACKING SETUP ===
    wandb.init(
        project="covid-tweet-sentiment-hf-bert-regularloss",
        name=f"trial_{trial.number}",
        config={
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "warmup_ratio": warmup_ratio,
            "weight_decay": weight_decay,
            "attention_dropout": attention_dropout,
            "hidden_dropout": hidden_dropout
            # "rdrop_alpha": rdrop_alpha ,
        },
        reinit=True
    )

    try:
        # === MODEL SETUP ===
        model = _setup_model(attention_dropout, hidden_dropout)

        # === TRAINING CONFIGURATION ===
        training_args = _create_training_args(
            trial_number=trial.number,
            learning_rate=learning_rate,
            batch_size=batch_size,
            num_epochs=num_epochs,
            warmup_ratio=warmup_ratio,
            weight_decay=weight_decay,
            label_smoothing_factor=label_smoothing
        )

        trial_params = {
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "warmup_ratio": warmup_ratio,
            "weight_decay": weight_decay,
            "attention_dropout": attention_dropout,
            "hidden_dropout": hidden_dropout
        }

        trainer = _create_trainer(model, training_args, trial.number, trial_params)
        trainer.train()

        # Set Checkpoint per trial
        checkpoint_dir = f"./checkpoints_bert/trial_{trial.number}"
        os.makedirs(checkpoint_dir, exist_ok=True)

        final_epoch = int(trainer.state.epoch)
        checkpoint_path = f"{checkpoint_dir}/final_epoch_{final_epoch}.ckpt"

        # Get final training loss
        final_loss = 0
        if trainer.state.log_history:
            for log_entry in reversed(trainer.state.log_history):
                if 'train_loss' in log_entry:
                    final_loss = log_entry['train_loss']
                    break

        eval_results = trainer.evaluate()
        current_score = eval_results["eval_quadratic_weighted_kappa"]

        save_training_checkpoint(
            model=trainer.model,
            optimizer=None,
            epoch=final_epoch,
            loss=final_loss,
            trial_params=trial_params,
            filepath=checkpoint_path,
            trial_number=trial.number,
            current_score=current_score,
            trainer=trainer
        )

        # Log GPU usage if available
        if device.type == "cuda":
            print(f"GPU Memory Used: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")

        return current_score

    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        raise optuna.exceptions.TrialPruned()

    finally:
        # === CLEANUP ===
        if device.type == "cuda":
            torch.cuda.empty_cache()
        wandb.finish()


def _setup_model(attention_dropout, hidden_dropout):
    """Setup and configure the model with dropout and freezing"""
    model = AutoModelForSequenceClassification.from_pretrained(
        "finiteautomata/bertweet-base-sentiment-analysis",
        num_labels=5,
        id2label=ordinal_id2label,
        label2id=ordinal_label2id,
        ignore_mismatched_sizes=True
    )

    # Apply dropout configuration
    model.config.attention_probs_dropout_prob = attention_dropout
    model.config.hidden_dropout_prob = hidden_dropout

    # GPU optimizations
    if device.type == "cuda":
        model.gradient_checkpointing_enable()

        model.to(device)

    return model


def _create_training_args(trial_number, learning_rate, batch_size, num_epochs, warmup_ratio, weight_decay, label_smoothing_factor=0.1):
    """Create optimized training arguments"""
    return TrainingArguments(
        # output_dir=f"./results/trial_{trial_number}",

        # Core training parameters
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,  # Larger eval batch
        learning_rate=learning_rate,
        label_smoothing_factor=label_smoothing_factor,

        # Learning rate scheduling
        lr_scheduler_type="cosine",
        warmup_ratio=warmup_ratio,

        # Optimization
        optim="adamw_torch",
        weight_decay=weight_decay,
        max_grad_norm=1.0,

        # Evaluation and saving
        eval_strategy="epoch",
        save_strategy="no",
        save_total_limit=1,
        # load_best_model_at_end=True,
        metric_for_best_model="eval_quadratic_weighted_kappa",
        greater_is_better=True,

        # Performance optimizations
        fp16=device.type == "cuda",
        # tf32=device.type == "cuda",
        dataloader_pin_memory=True,
        dataloader_persistent_workers=device.type == "cuda",
        dataloader_num_workers=2 if device.type == "cuda" else 0,
        dataloader_drop_last=False,
        group_by_length=True,
        gradient_accumulation_steps=1,
        dataloader_prefetch_factor=2 if device.type == "cuda" else None,

        # Logging
        logging_steps=100,
        report_to="wandb",
        remove_unused_columns=False,

        # Evaluation optimizations
        eval_accumulation_steps=None,
        prediction_loss_only=False,

    )

def _create_trainer(model, training_args, trial_number, trial_params):
    """ordinal loss trainer"""

    callbacks = [
        EarlyStoppingCallback(early_stopping_patience=2),
        SimpleMetricsLogger(),
        ]

    return Trainer(  # Uses ordinal loss always
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_detailed_metrics,
        tokenizer=tokenizer,
        callbacks=callbacks,
    )

In [20]:
from datetime import datetime
import json

def save_best_hyperparameters(study, model_name="bertweet_base"):
    """Save the best hyperparameters found by Optuna"""

    # Check if any trial completed successfully
    completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    if not completed_trials:
        print("No completed trials found. Skipping saving best hyperparameters.")
        return None

    best_params = study.best_params
    best_score = study.best_value

    # Save to JSON file
    results = {
        "model_name": model_name,
        "best_score": best_score,
        "best_params": best_params,
        "timestamp": str(datetime.now()),
        "total_trials": len(study.trials),
        "completed_trials": len(completed_trials)
    }

    filename = f"best_params_{model_name}.json"
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)

    print(f"Best hyperparameters saved to {filename}")
    print(f"Best score: {best_score:.4f}")
    print(f"Best params: {best_params}")
    print(f"Completed {len(completed_trials)}/{len(study.trials)} trials")

    return results

## Training

In [21]:
# Run optimization

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=8)

bert_results = save_best_hyperparameters(study, "bertweet_base")


[I 2025-08-01 07:39:41,030] A new study created in memory with name: no-name-13c69f66-8008-465f-bdf4-85ddf2eaf009


TRIAL 0 - TESTING THESE PARAMETERS:
Learning Rate:      3.36e-05
Epochs:             12
Warmup Ratio:       0.079
Weight Decay:       0.077
Attention Dropout:  0.301
Hidden Dropout:     0.327


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1318007737.py:216: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(  # Uses ordinal loss always
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parame

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,F1 Extremely Negative,Precision Extremely Negative,Recall Extremely Negative,Accuracy Extremely Negative,F1 Negative,Precision Negative,Recall Negative,Accuracy Negative,F1 Neutral,Precision Neutral,Recall Neutral,Accuracy Neutral,F1 Positive,Precision Positive,Recall Positive,Accuracy Positive,F1 Extremely Positive,Precision Extremely Positive,Recall Extremely Positive,Accuracy Extremely Positive,Mae,Adjacent Accuracy,Quadratic Weighted Kappa
1,1.128500,1.017624,0.711269,0.711390,0.743204,0.712159,0.760246,0.760246,0.760246,0.760246,0.666316,0.641700,0.692896,0.692896,0.771612,0.734010,0.813273,0.813273,0.683327,0.651494,0.718431,0.718431,0.710938,0.928571,0.575949,0.575949,0.359422,0.934588,0.833697
2,0.911400,0.890793,0.798715,0.798142,0.795248,0.815284,0.795160,0.687593,0.942623,0.942623,0.719491,0.764128,0.679781,0.679781,0.851088,0.866978,0.835771,0.835771,0.785091,0.793905,0.776471,0.776471,0.852564,0.863636,0.841772,0.841772,0.249943,0.956162,0.894932
3,0.778300,0.852766,0.817535,0.818170,0.839895,0.803928,0.788372,0.911290,0.694672,0.694672,0.772679,0.754953,0.791257,0.791257,0.867991,0.902795,0.835771,0.835771,0.802651,0.756419,0.854902,0.854902,0.858247,0.874016,0.843038,0.843038,0.226303,0.958228,0.898681
4,0.725200,0.895241,0.810879,0.810341,0.816922,0.824504,0.833333,0.807692,0.860656,0.860656,0.775185,0.808789,0.744262,0.744262,0.872226,0.934447,0.817773,0.817773,0.769604,0.793995,0.746667,0.746667,0.832965,0.739686,0.953165,0.953165,0.230663,0.960982,0.906755
5,0.682500,0.938338,0.796420,0.795675,0.805892,0.815249,0.836257,0.797398,0.879098,0.879098,0.775652,0.825926,0.731148,0.731148,0.856624,0.926702,0.796400,0.796400,0.742834,0.777207,0.711373,0.711373,0.810493,0.702226,0.958228,0.958228,0.243516,0.963277,0.904482
6,0.640100,0.867103,0.835437,0.835753,0.833765,0.842702,0.838710,0.780919,0.905738,0.905738,0.776087,0.771892,0.780328,0.780328,0.854848,0.872365,0.838020,0.838020,0.837998,0.842314,0.833725,0.833725,0.877922,0.901333,0.855696,0.855696,0.193023,0.972688,0.923897
7,0.610900,0.917472,0.823502,0.822853,0.815695,0.837953,0.804233,0.705882,0.934426,0.934426,0.745958,0.790698,0.706011,0.706011,0.861853,0.871264,0.852643,0.852643,0.822437,0.847049,0.799216,0.799216,0.880199,0.863581,0.897468,0.897468,0.208400,0.970163,0.918732
8,0.592900,0.904674,0.826027,0.825156,0.830274,0.827575,0.830472,0.871622,0.793033,0.793033,0.781232,0.809133,0.755191,0.755191,0.840737,0.810867,0.872891,0.872891,0.816294,0.831570,0.801569,0.801569,0.869513,0.828179,0.915190,0.915190,0.200367,0.975442,0.920538


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 4: Train Loss: 1.2908, Val Loss: 1.8073, Val F1: 0.0000, QWK: 0.0000
✅ Trial checkpoint saved: ./checkpoints_bert/trial_7/final_epoch_4.ckpt
✅ Model files saved in: ./checkpoints_bert/trial_7
📊 Trial 7 score: 0.0000 (Best: 0.9284)
GPU Memory Used: 4.13 GB


Epoch,▁▃▆██
Learning_Rate,▁▁▁▁▁
Stage,▁▁▁▁▁
Train Accuracy,▁▁▁▁▁
Train Loss,▁▁▁▁█
Unfrozen_Layers,▁▁▁▁▁
Validation Accuracy,██▃▁▁
Validation Adjacent Accuracy,▇█▅▁▁
Validation F1,▁▁▁▁▁
Validation Loss,▂▁▇██
Validation MAE,▂▁▆██


[I 2025-08-01 09:05:49,368] Trial 7 finished with value: 0.0 and parameters: {'learning_rate': 0.0003897773320704888, 'batch_size': 32, 'label_smoothing': 0.08451958274914737, 'num_epochs': 14, 'warmup_ratio': 0.06030116339682273, 'weight_decay': 0.14619808960138364, 'attention_dropout': 0.37166988585343874, 'hidden_dropout': 0.3669526106323674}. Best is trial 4 with value: 0.9284241099301461.


Best hyperparameters saved to best_params_bertweet_base.json
Best score: 0.9284
Best params: {'learning_rate': 0.00021229985961337253, 'batch_size': 32, 'label_smoothing': 0.11524364144454653, 'num_epochs': 11, 'warmup_ratio': 0.13273796418047804, 'weight_decay': 0.1121797776236029, 'attention_dropout': 0.33099760772482156, 'hidden_dropout': 0.38867822735790303}
Completed 8/8 trials


In [22]:
import time
import os

def calculate_model_metrics(model_path="./best_model_so_far"):
    """Calculate inference time and model size"""

    # 1. Calculate Model Size
    def get_model_size_mb(path):
        total_size = 0
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                total_size += os.path.getsize(filepath)
        return total_size / (1024 * 1024)  # Convert to MB

    model_size_mb = get_model_size_mb(model_path)

    # 2. Calculate Inference Time
    def measure_inference_time():
        # Load model for timing
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        tokenizer_for_timing = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")

        if device.type == "cuda":
            model = model.to(device)
        model.eval()

        # Sample text for timing
        sample_text = "COVID vaccines are helping to reduce hospitalizations significantly."

        # Warm-up runs (don't count these)
        for _ in range(3):
            inputs = tokenizer_for_timing(sample_text, return_tensors="pt", truncation=True, padding=True)
            if device.type == "cuda":
                inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                _ = model(**inputs)

        # Actual timing runs
        times = []
        num_runs = 10

        for _ in range(num_runs):
            inputs = tokenizer_for_timing(sample_text, return_tensors="pt", truncation=True, padding=True)
            if device.type == "cuda":
                inputs = {k: v.to(device) for k, v in inputs.items()}

            start_time = time.time()
            with torch.no_grad():
                _ = model(**inputs)
            end_time = time.time()

            times.append(end_time - start_time)

        # Return average inference time
        avg_inference_time = sum(times) / len(times)
        return avg_inference_time

    inference_time = measure_inference_time()

    return model_size_mb, inference_time

# Load the best model
best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)

# Create trainer for test evaluation
test_trainer = Trainer(
    model=best_model,
    args=TrainingArguments(
        output_dir="./temp",
        per_device_eval_batch_size=16,
        remove_unused_columns=False,
    ),
    tokenizer=tokenizer,
    compute_metrics=compute_detailed_metrics,  # Use the enhanced function!
)

# Evaluate on test set
test_results = test_trainer.evaluate(test_dataset)

# Calculate performance metrics
model_size_mb, inference_time_sec = calculate_model_metrics(best_model_path)

print("FINAL TEST SET EVALUATION - COMPREHENSIVE ANALYSIS")

# Standard Classification Metrics
print(test_results)
print("\nOVERALL CLASSIFICATION METRICS:")
print(f"Accuracy:           {test_results['eval_accuracy']:.4f}")
print(f"Validation QWK:     {test_results['eval_quadratic_weighted_kappa']:.4f}")
print(f"F1:                 {test_results['eval_f1']:.4f}")
print(f"Precision-Macro:    {test_results['eval_precision']:.4f}")
print(f"Recall-Macro:       {test_results['eval_recall']:.4f}")

# Ordinal-Aware Metrics
print("\nORDINAL-AWARE METRICS:")
print(f"Mean Absolute Error:        {test_results['eval_mae']:.4f}")
print(f"Adjacent Accuracy:          {test_results['eval_adjacent_accuracy']:.4f}")
print(f"Quadratic Weighted Kappa:   {test_results['eval_quadratic_weighted_kappa']:.4f}")

# Performance Metrics
print("\nPERFORMANCE METRICS:")
print(f"Inference Time:             {inference_time_sec:.4f} sec")
print(f"Model Size:                 {model_size_mb:.1f} MB")

# Per-Class Detailed Analysis (PRESERVED)
print("\nPER-CLASS PERFORMANCE ANALYSIS:")
sentiment_classes = ["extremely_negative", "negative", "neutral", "positive", "extremely_positive"]
class_display_names = ["Extremely Negative", "Negative", "Neutral", "Positive", "Extremely Positive"]

for i, (class_key, class_name) in enumerate(zip(sentiment_classes, class_display_names)):
    print(f"\n{class_name}:")
    print(f"  F1-Score:   {test_results.get(f'eval_f1_{class_key}', 0):.4f}")
    print(f"  Precision:  {test_results.get(f'eval_precision_{class_key}', 0):.4f}")
    print(f"  Recall:     {test_results.get(f'eval_accuracy_{class_key}', 0):.4f}")
    print(f"  Accuracy:   {test_results.get(f'eval_accuracy_{class_key}', 0):.4f}")

# Performance Analysis (PRESERVED + ENHANCED)
print("\nPERFORMANCE INSIGHTS:")
print(f"• MAE {test_results['eval_mae']:.2f}: On average off by {test_results['eval_mae']:.2f} sentiment levels")
print(f"• Adjacent Accuracy {test_results['eval_adjacent_accuracy']:.1%}: Predictions within 1 sentiment level")
print(f"• QWK {test_results['eval_quadratic_weighted_kappa']:.3f}: {'Excellent' if test_results['eval_quadratic_weighted_kappa'] > 0.8 else 'Good' if test_results['eval_quadratic_weighted_kappa'] > 0.6 else 'Moderate'} ordinal agreement")
# Added inference speed and efficiency insights
print(f"• Inference Speed: {1/inference_time_sec:.1f} predictions per second")
print(f"• Model Efficiency: {model_size_mb:.1f} MB storage required")

# Class Performance Analysis (PRESERVED)
f1_scores = [test_results.get(f'eval_f1_{class_key}', 0) for class_key in sentiment_classes]
best_class_idx = f1_scores.index(max(f1_scores))
worst_class_idx = f1_scores.index(min(f1_scores))

print(f"\nCLASS-SPECIFIC INSIGHTS:")
print(f"• Best performing class: {class_display_names[best_class_idx]} (F1: {f1_scores[best_class_idx]:.4f})")
print(f"• Most challenging class: {class_display_names[worst_class_idx]} (F1: {f1_scores[worst_class_idx]:.4f})")

# COVID-specific insights (PRESERVED)
extreme_avg = (f1_scores[0] + f1_scores[4]) / 2  # extremely negative + extremely positive
moderate_avg = (f1_scores[1] + f1_scores[3]) / 2  # negative + positive
neutral_score = f1_scores[2]

print(f"\nCOVID SENTIMENT INSIGHTS:")
print(f"• Extreme emotions (avg F1: {extreme_avg:.3f}): {'Challenging' if extreme_avg < 0.7 else 'Well-handled'}")
print(f"• Moderate emotions (avg F1: {moderate_avg:.3f}): {'Needs work' if moderate_avg < 0.75 else 'Good performance'}")
print(f"• Neutral sentiment (F1: {neutral_score:.3f}): {'Difficult to detect' if neutral_score < 0.8 else 'Well-identified'}")

# Final Summary with Key Metrics
print(f"\n📊 FINAL SUMMARY:")
print(f"F1: {test_results['eval_f1']:.4f} | QWK: {test_results['eval_quadratic_weighted_kappa']:.4f} | Inference: {inference_time_sec:.4f}s | Size: {model_size_mb:.1f}MB")

/tmp/ipython-input-3255646991.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  test_trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


FINAL TEST SET EVALUATION - COMPREHENSIVE ANALYSIS
{'eval_loss': 0.6327186822891235, 'eval_model_preparation_time': 0.0037, 'eval_accuracy': 0.8233060747663551, 'eval_f1': 0.8223617028680927, 'eval_precision': 0.8209153033370565, 'eval_recall': 0.8357680469803682, 'eval_f1_extremely_negative': 0.8286384976525821, 'eval_precision_extremely_negative': 0.7986425339366516, 'eval_recall_extremely_negative': 0.8609756097560975, 'eval_accuracy_extremely_negative': 0.8609756097560975, 'eval_f1_negative': 0.7915869980879541, 'eval_precision_negative': 0.8214285714285714, 'eval_recall_negative': 0.7638376383763837, 'eval_accuracy_negative': 0.7638376383763837, 'eval_f1_neutral': 0.8396624472573839, 'eval_precision_neutral': 0.7824377457404981, 'eval_recall_neutral': 0.9059180576631259, 'eval_accuracy_neutral': 0.9059180576631259, 'eval_f1_positive': 0.8100278551532033, 'eval_precision_positive': 0.8644470868014269, 'eval_recall_positive': 0.7620545073375262, 'eval_accuracy_positive': 0.762054507